In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
articles_df = pd.read_csv('./data/shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']

interactions_df = pd.read_csv('./data/users_interactions.csv')

interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

interactions_df['eventStrength'] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]

interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].last()
)
        
interactions_full_df = interactions_full_df.reset_index()


split_ts = 1475519545
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()


# =======================

In [4]:
interactions_full_df.head()

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


#### ЗАДАНИЕ 6.1
<a name="6.1"> </a>

In [40]:
ratings = pd.pivot_table(
    interactions_train_df,
    values='eventStrength',
    index='personId',
    columns='contentId').fillna(0)
ratings.loc['-1032019229384696495','943818026930898372']

2.321928094887362

#### ЗАДАНИЕ 6.2

In [6]:
ratings_m = ratings.values
ratings_m.mean()

0.016673743043640697

#### ЗАДАНИЕ 6.3

In [7]:
similarity_users = np.zeros((len(ratings_m), len(ratings_m)))
for i in (range(len(ratings_m)-1)):
    for j in range(i+1, len(ratings_m)):
     
        mask_uv = (ratings_m[i] != 0) & (ratings_m[j] != 0)
        ratings_v = ratings_m[i, mask_uv]
        ratings_u = ratings_m[j, mask_uv]

        similarity_users[i,j] = np.corrcoef(ratings_v, ratings_u)[0, 1]
        similarity_users[j,i] = similarity_users[i,j]

similarity_users[3,40]

C:\Users\Manych\anaconda3\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\Manych\anaconda3\lib\site-packages\numpy\core\_methods.py:180: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
C:\Users\Manych\anaconda3\lib\site-packages\numpy\lib\function_base.py:2634: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\Manych\anaconda3\lib\site-packages\numpy\lib\function_base.py:2493: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\Users\Manych\anaconda3\lib\site-packages\numpy\lib\function_base.py:2493: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
C:\Users\Manych\anaconda3\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\Manych\anaconda3\lib\site-packages\numpy\lib\function_b

-0.3333333333333333

In [8]:
interactions = (
    interactions_train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)
 
interactions['true_test'] = (
    interactions_test_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)
 
interactions['true_test'] = [ [] if x is np.NaN else x for x in interactions['true_test'] ]

prediction_user_based = []
for i in range(len(similarity_users)):
    users_sim = similarity_users[i] > 0
    if not any(users_sim):
        prediction_user_based.append([])
    else:
        tmp_recommend = np.argsort(ratings_m[users_sim].sum(axis=0))[::-1]
        tmp_recommend = ratings.columns[tmp_recommend]
        recommend = np.array(tmp_recommend)[~np.in1d(tmp_recommend, interactions.iloc[i])][:10]
        prediction_user_based.append(list(recommend))
interactions['prediction_user_based'] = prediction_user_based
prediction_user_based[35][0]

C:\Users\Manych\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


'8657408509986329668'

После того как сделаны предсказания, можно вычислить качество по метрике, которую мы определили в предыдущем модуле при решении этой задачи:
```python
def calc_precision(column):
    return ( interactions.apply(  lambda row:len(set(row['true_test']).intersection(
                set(row[column]))) /min(len(row['true_test']) + 0.001, 10.0), axis=1)).mean()
```

In [34]:
def calc_precision(column):
    return ( interactions.apply(  lambda row:len(set(row['true_test']).intersection(
                set(row[column]))) /min(len(row['true_test']) + 0.001, 10.0), axis=1)).mean()

### Задание 6.5
Вычислите точность полученного предсказания. Ответ округлите до трёх знаков после точки-разделителя.

In [35]:
round(calc_precision('prediction_user_based'), 3)

0.003

### Задание 6.6
Теперь реализуем рекомендательную систему с использованием **SVD**.

Разложите матрицу взаимодействий пользователей со статьями с помощью функции `svd` из модуля `scipy`. Найдите максимальное значение в получившейся матрице **U**. Результат округлите до двух знаков после точки-разделителя.

In [20]:
from scipy.linalg import svd

U, sigma, V = svd(ratings)
ans = U.max()
round(ans,2)

0.71

In [37]:
# U.shape, sigma.shape, V.shape

>Значения матрицы с сингулярными числами отсортированы по убыванию. Допустим, мы хотим оставить только первые 100 компонент и получить скрытые представления размерности 100. Для этого необходимо оставить 100 столбцов в матрице U, только первые 100 значений из sigma (и сделать из них диагональную матрицу) и 100 столбцов в матрице V. Затем необходимо перемножить преобразованные матрицы.

### Задание 6.7
Найдите сумму всех элементов в новой сингулярной матрице. Ответ округлите до двух знаков после точки-разделителя.

In [23]:
k = 100
s = np.diag(sigma[:k])
U = U[:, 0:k]
V = V[0:k, :]

round(s.sum(), 2)

2096.53

In [36]:
# U.shape, sigma.shape, V.shape

Теперь мы можем сделать предсказание по полученной матрице.

> Примечание. Помните, что не нужно учитывать статьи, которые уже были просмотрены пользователем.

Найдите для каждого пользователя статьи с наибольшими оценками в восстановленной матрице.

### Задание 6.8
Вычислите качество полученного предсказания, используя всё ту же метрику точности. Ответ округлите до трёх знаков после точки-разделителя.

In [42]:
new_ratings = pd.DataFrame(
    U.dot(s).dot(V), index=ratings.index, columns=ratings.columns
)
top_k = 10
predictions = []

for personId in interactions.index:
    prediction = (
        new_ratings.loc[personId].sort_values(ascending=False).index.values
    )

    predictions.append(
        list(
            prediction[
                ~np.in1d(prediction, interactions.loc[personId, 'true_train'])
            ]
        )[:top_k]
    )

interactions['prediction_svd'] = predictions

round(calc_precision('prediction_svd'), 3)

0.012

### Задание 6.9
Возьмите матрицу, подготовленную в [задании 6.1](#6.1) задании 6.1. Преобразуйте её в разреженную матрицу:

```python
from scipy.sparse import csr_matrix
ratings_matrix = csr_matrix(ratings)
```
Воспользовавшись функцией `random_train_test_split()` из библиотеки `lightfm`, разделите данные на валидационную и обучающую выборки в соотношении **1:2** (30% на валидационную выборку, 70% на обучающую). В качестве значения параметра `random_state` возьмите число **13**.

Обучите модель `LightFM` со **100** компонентами, параметром `random_state` = **13**, темпом обучения **0.05** и функцией потерь **'warp'**. Обратите внимание на то, что так как в данном случае у нас нет  **item -признаков**, то параметр `item_features` задавать не нужно.

Вычислите показатель точности (`precision@k`) при **k = 10**. Ответ округлите до **двух знаков после точки-разделителя**.

In [43]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k  
from scipy.sparse import csr_matrix

ratings_matrix = csr_matrix(ratings) # передаём в качестве аргументов в функцию 
                                     # выставленный рейтинг (это будут значения матрицы), 
                                     # а также id пользователя и id книги (это будут 
                                     # индексы для строк и столбцов матрицы)
model = LightFM(loss='warp', # определяем функцию потерь
                random_state=13, # фиксируем случайное разбиение
                learning_rate=0.05, # темп обучения
                no_components=100) # размерность вектора для представления данных в модели

train,test = random_train_test_split(ratings_matrix, test_percentage=0.3, random_state=13)
model.fit(train)
prec_score = precision_at_k(model, test).mean()

print(round(prec_score, 2))

C:\Users\Manych\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


0.04
